In [1]:
import streamlit as st
import jsonlines
import random
import pandas as pd
import seaborn as sns
import json
import fileinput
import numpy as np
from pprint import pprint
from wordcloud import WordCloud
from textblob_de import TextBlobDE as TextBlob
import re
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict
from datetime import date
import re
import string
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

2022-06-08 10:03:02.363 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
#Importing Dataset
olafScholzJsonLines = jsonlines.open("OlafScholz.jl")

olafScholzTwitter = pd.read_json("OlafScholz.jl", lines = True)
print("Datensatzlänge: " + str(olafScholzTwitter.info()))

print(olafScholzTwitter.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   http_status   65 non-null     int64 
 1   account_name  65 non-null     object
 2   account_data  65 non-null     object
 3   params        65 non-null     object
 4   response      65 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.7+ KB
Datensatzlänge: None
   http_status account_name  \
0          200   OlafScholz   
1          200   OlafScholz   
2          200   OlafScholz   
3          200   OlafScholz   
4          200   OlafScholz   

                                        account_data  \
0  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
1  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
2  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
3  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
4  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   

         

In [3]:
# Reply Dataframe
# Importing Data into final Dataframe for ML
df = pd.DataFrame(columns = ["datetime", "text"])
iterator = 0

for line in olafScholzJsonLines:
    keyResponse = line["response"]
    data = keyResponse["data"]
    newDataRow = [None, None]
    userName = line["account_name"]
    keyResponse = line["response"]
    data = keyResponse["data"]


    for tweet in data:

        tweetTarget = 1
       
        tweetDate = tweet["created_at"][0:10]+" "+tweet["created_at"][11:19]
        tweetText = tweet["text"]  
        
        newDataRow[0] = tweetDate
        newDataRow[1] = tweetText
        df.loc[len(df)] = newDataRow
    iterator += 1

print(df.head())

              datetime                                               text
0  2022-04-17 07:11:52  RT @Bundeskanzler: Ich wünsche Ihnen und Ihren...
1  2022-04-14 16:40:32  RT @Bundeskanzler: Meine Solidarität gilt @Kar...
2  2022-04-13 17:16:53  RT @Bundeskanzler: Sie haben Hilfe organisiert...
3  2022-04-13 14:58:53  RT @Bundeskanzler: Verantwortung übernehmen he...
4  2022-04-12 18:59:51  RT @Bundeskanzler: Die Sanktionen gegen #Russl...


In [8]:
#cleansedTweetDF = replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:") or replyDataFrame["text"].str.contains("!RT @") ]
print("Size of dataframe :\n"+ str(df.count())+"\n----")
maskBundesKanz = df["text"].str.contains("RT @Bundeskanzler:")
maskNoRT = df["text"].str.contains("RT @")==False
print("Size of dataframe with RT @Bundeskanzler:\n" + str(df[maskBundesKanz].count())+"\n----")
print("Size of dataframe without RT @:\n" + str(df[maskNoRT].count())+"\n----")
#cleansedTweetDF contains all Tweets posted directly by Olaf before or after being Kanzler
cleansedTweetDF = df[maskBundesKanz | maskNoRT]
print("Size of cleansedTweetDF @:\n" + str(cleansedTweetDF.count())+"\n----")
print(df.head())
#cleansedTweetDF has "RT @Bundeskanzler:" still in the tweet. To not have any issues with ML we removed that part of the Tweet in the following parts

Size of dataframe :
datetime    6419
text        6419
dtype: int64
----
Size of dataframe with RT @Bundeskanzler:
datetime    48
text        48
dtype: int64
----
Size of dataframe without RT @:
datetime    4071
text        4071
dtype: int64
----
Size of cleansedTweetDF @:
datetime    4119
text        4119
dtype: int64
----
              datetime                                               text
0  2022-04-17 07:11:52  RT @Bundeskanzler: Ich wünsche Ihnen und Ihren...
1  2022-04-14 16:40:32  RT @Bundeskanzler: Meine Solidarität gilt @Kar...
2  2022-04-13 17:16:53  RT @Bundeskanzler: Sie haben Hilfe organisiert...
3  2022-04-13 14:58:53  RT @Bundeskanzler: Verantwortung übernehmen he...
4  2022-04-12 18:59:51  RT @Bundeskanzler: Die Sanktionen gegen #Russl...


In [9]:
# Removing the Text \"RT @Bundeskanzler:\" from the text\n
print("ReplyDataframe: \n" + str(df[maskBundesKanz].count()))
cleansedAtBundKanzDF = df[maskBundesKanz].copy()
cleansedAtBundKanzDF.text = df.text.str.strip("RT @Bundeskanzler: ")
print(cleansedAtBundKanzDF.head())  

ReplyDataframe: 
datetime    48
text        48
dtype: int64
              datetime                                               text
0  2022-04-17 07:11:52  Ich wünsche Ihnen und Ihren Familien frohe #Os...
1  2022-04-14 16:40:32  Meine Solidarität gilt @Karl_Lauterbach: Jeder...
2  2022-04-13 17:16:53  Sie haben Hilfe organisiert, Kräfte gebündelt ...
3  2022-04-13 14:58:53  Verantwortung übernehmen heißt: Deutschland li...
4  2022-04-12 18:59:51  Die Sanktionen gegen #Russland sind notwendig ...
